## 1º FUENTE DE DATOS: KAGGLE European Database Soccer

- Esta primera fuente de datos se obtiene de https://www.kaggle.com/datasets/hugomathien/soccer

La información que nos da es de :
 1. Más de 25,000 partidos
 2. Más de 10,000 futbolistas
 3. 11 paises europeos , aprox. 300 equipos
 4. Temporadas que van desde 2008 a 2016
 6. Atributos para jugadores y equipos obtenidos del videojuego EA Sports' FIFA 
 7. Alineaciones detalladas para cada partido con la posición de cada jugador
 8. Distintos eventos de unos 10000 de esos partidos (goles, tarjetas, corners, disparos a puerta,...)
 9. Información de casas de apuestas
 
 
Se han hecho pruebas para cargarlos los datos en un Dataframe a partir de la exportación de cada tabla de la BD SQLite a un archivo CSV por cada tabla, posteriormente se ha leído cada CSV cargando en un Dataframe. 

Esto ha entrañado ciertas dificultades porque además del propio trabajo de exportar las tablas y leerlas, se han visto problemas en la tabla Match (la más grande) en la que una sola línea que tenía algunos campos XML con retornos de carro, provocaba que se rompiera la estructura. De todos modos, se ha podido avanzar viendo como se iba a hacer join de los datos y decidiendo que columnas de la tabla se iban a eliminar. 

Conociendo eso, se comienza de nuevo pero se cargan directamente las tablas con sqlite3 que facilita mucho todo el trabajo.

In [5]:
import sqlite3
import pandas as pd
import numpy as np

Teniendo en la misma ruta que este notebook, la Base de Datos SQLite, realizamos la conexión con la librería sqlite3 de Python

In [6]:
pathSqlite = './EuropeanSoccerDB.sqlite'
cnx = sqlite3.connect(pathSqlite)

Leemos la Tabla de paises usando la consulta de queries de Pandas.

In [7]:
df_country =  pd.read_sql_query("SELECT id, name as country \
FROM Country", cnx)

Leemos la Tabla de ligas europeas con la misma longitud que el de paises, 11 filas.  
Pero ahora omitimos en la carga la columna 'country_id' que es redundante, (tiene el mismo valor que 'id' ).  
NOTA: También con pd.read_csv se puede hacer ésto, usando el campo 'names'.

In [8]:
df_league = pd.read_sql_query("SELECT id, name as league FROM League", cnx)

Realizamos un join para tener un solo dataframe

In [9]:
df_country_league = pd.merge(df_country, df_league, on=['id'], how='inner', sort=True)

Mostramos la información de las columnas y los tipos que tienen

In [10]:
df_country_league.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       11 non-null     int64 
 1   country  11 non-null     object
 2   league   11 non-null     object
dtypes: int64(1), object(2)
memory usage: 392.0+ bytes


In [11]:
#Convertimos a tipo string las dos que son object
df_country_league['country'] = df_country_league['country'].astype('string')
df_country_league['league'] = df_country_league['league'].astype('string')

In [12]:
#Comprobamos el cambio
df_country_league.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       11 non-null     int64 
 1   country  11 non-null     string
 2   league   11 non-null     string
dtypes: int64(1), string(2)
memory usage: 392.0 bytes


Leemos la Tabla de equipos, pero sólo dos columnas.

In [13]:
df_team = pd.read_sql_query("SELECT team_api_id, team_long_name as team FROM Team", cnx)

In [14]:
#Vemos la información del Dataframe
df_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   team_api_id  299 non-null    int64 
 1   team         299 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.8+ KB


In [15]:
# Convertimos el nombre del equipo a String.
df_team['team'] = df_team['team'].astype('string')

Respecto a la Tabla Match, hacemos una selección de las columnas que nos interesan, ya que 115 son muchas características.
Ya tenemos pensado que las 30 últimas, que corresponden a casas de apuestas, no nos interesan.
También se decide descartar las columnas: 'shotoff', 'cross', 'corner', 'possession'.

In [16]:
df_match = pd.read_sql_query("SELECT country_id,season,stage,date,match_api_id, \
home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2, \
home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,\
home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,\
away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,\
away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,\
home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,\
home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,\
away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,\
home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,\
home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,\
away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,\
away_player_10,away_player_11,goal,shoton,foulcommit,card FROM Match", cnx)

Revisamos la información de las columnas que se han cargado, con el tipo de dato y cuantos valores nulos tenemos.

In [17]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 79 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country_id        25979 non-null  int64  
 1   season            25979 non-null  object 
 2   stage             25979 non-null  int64  
 3   date              25979 non-null  object 
 4   match_api_id      25979 non-null  int64  
 5   home_team_api_id  25979 non-null  int64  
 6   away_team_api_id  25979 non-null  int64  
 7   home_team_goal    25979 non-null  int64  
 8   away_team_goal    25979 non-null  int64  
 9   home_player_X1    24158 non-null  float64
 10  home_player_X2    24158 non-null  float64
 11  home_player_X3    24147 non-null  float64
 12  home_player_X4    24147 non-null  float64
 13  home_player_X5    24147 non-null  float64
 14  home_player_X6    24147 non-null  float64
 15  home_player_X7    24147 non-null  float64
 16  home_player_X8    24147 non-null  float6

El problema que se aprecia es la cantidad de filas nulas en las 4 últimas columnas, unas 11K, 
eliminar todas las filas con algún valor nulo, supone perder más del 40% de la BD.<br>
Se decide rellenar con la cadena vacía estas 4 columnas.

In [18]:
df_match['goal'] = df_match['goal'].fillna('')
df_match['shoton'] = df_match['shoton'].fillna('')
df_match['foulcommit'] = df_match['foulcommit'].fillna('')
df_match['card'] = df_match['card'].fillna('')

In [19]:
# Se hacen varias conversiones de tipos, una importante es la conversión del día del partido a tipo
# fecha
df_match['country_id'] = df_match['country_id'].astype('uint32')
df_match['season'] = df_match['season'].astype('string')
df_match['stage'] = df_match['stage'].astype('uint8')
df_match['home_team_goal'] = df_match['home_team_goal'].astype('uint8')
df_match['away_team_goal'] = df_match['away_team_goal'].astype('uint8')
df_match['date'] = pd.to_datetime(df_match['date'])

In [20]:
#Manera de averiguar cuantas filas tienen un valor nulo en una característica determinada 
nan_rows = df_match[df_match['home_player_X1'].isnull()]
print(len(nan_rows.index))

1821


In [21]:
# Se realizan conversiones de tipos
# Para las columnas de nombre '[home|away]_player_[X|Y]{numero}' se elige un float16 que es suficiente para 
# guardar numeros del 1 al 11, no elegimos un int porque no admite nulos, no se los hemos quitado
# Para las columnas '[home|away]_player_{numero}' que contienen identificadores, utilizamos el tipo
# Int64 que es más adecuado que el float y en este caso (la I es mayúscula), si admite nulos.
def convertType(fieldname, typename):
    for i in np.arange(1,12):
        if not df_match[fieldname+str(i)].empty:
            df_match[fieldname+str(i)] = df_match[fieldname+str(i)].astype(typename) 
            
convertType('home_player_X', 'float16')
convertType('home_player_Y', 'float16')
convertType('away_player_X', 'float16')
convertType('away_player_Y', 'float16')
convertType('home_player_', 'Int64')
convertType('away_player_', 'Int64')               

In [22]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 79 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   country_id        25979 non-null  uint32        
 1   season            25979 non-null  string        
 2   stage             25979 non-null  uint8         
 3   date              25979 non-null  datetime64[ns]
 4   match_api_id      25979 non-null  int64         
 5   home_team_api_id  25979 non-null  int64         
 6   away_team_api_id  25979 non-null  int64         
 7   home_team_goal    25979 non-null  uint8         
 8   away_team_goal    25979 non-null  uint8         
 9   home_player_X1    24158 non-null  float16       
 10  home_player_X2    24158 non-null  float16       
 11  home_player_X3    24147 non-null  float16       
 12  home_player_X4    24147 non-null  float16       
 13  home_player_X5    24147 non-null  float16       
 14  home_player_X6    2414

Hacemos un mergeo con la tabla de Paises y ligas, eliminamos la columna 'id' de ésta tabla, una vez usada como pivote, ya que es redundante.

In [23]:
df_match_league = pd.merge(df_country_league, df_match, left_on='id',right_on='country_id', how='inner')
df_match_league.drop("id", axis=1, inplace=True)

Leemos la Tabla de Futbolistas usando la consulta de queries de Pandas.

In [24]:
df_player =  pd.read_sql_query("SELECT player_api_id, player_name, birthday, height, weight \
FROM Player", cnx)

In [25]:
#Los datos del Dataframe indican que no hay nulos, y que se dispone de una BD de unos 11K futbolistas
df_player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_api_id  11060 non-null  int64 
 1   player_name    11060 non-null  object
 2   birthday       11060 non-null  object
 3   height         11060 non-null  int64 
 4   weight         11060 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 432.2+ KB


In [26]:
#Convertimos los tipos a nuestra conveniencia
df_player['player_name'] = df_player['player_name'].astype('string')
df_player['birthday'] = pd.to_datetime(df_player['birthday'])
df_player['height'] = df_player['height'].astype(np.uint8)
df_player['weight'] = df_player['weight'].astype(np.uint8)

 Vemos que la tabla estaba ordenada alfabeticamente por nombre del jugador.

In [27]:
df_player.head(5)

,player_api_id,player_name,birthday,height,weight
0,505942,Aaron Appindangoye,1992-02-29,182,187
1,155782,Aaron Cresswell,1989-12-15,170,146
2,162549,Aaron Doran,1991-05-13,170,163
3,30572,Aaron Galindo,1982-05-08,182,198
4,23780,Aaron Hughes,1979-11-08,182,154


Leemos la tabla de atributos de los futbolistas, seleccionando las columnas que nos interesan.

In [28]:
df_player_attributes = pd.read_sql_query("SELECT player_api_id, date, overall_rating, potential, \
preferred_foot, crossing, finishing, heading_accuracy, \
short_passing, volleys, dribbling, curve, free_kick_accuracy, long_passing,ball_control, \
acceleration, sprint_speed, agility, reactions, balance, shot_power, jumping, stamina, strength, \
long_shots, aggression, interceptions, positioning, vision, penalties, marking, standing_tackle, \
sliding_tackle, gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes \
FROM Player_Attributes", cnx)

In [29]:
#Vemos la información de esta tabla
df_player_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   player_api_id       183978 non-null  int64  
 1   date                183978 non-null  object 
 2   overall_rating      183142 non-null  float64
 3   potential           183142 non-null  float64
 4   preferred_foot      183142 non-null  object 
 5   crossing            183142 non-null  float64
 6   finishing           183142 non-null  float64
 7   heading_accuracy    183142 non-null  float64
 8   short_passing       183142 non-null  float64
 9   volleys             181265 non-null  float64
 10  dribbling           183142 non-null  float64
 11  curve               181265 non-null  float64
 12  free_kick_accuracy  183142 non-null  float64
 13  long_passing        183142 non-null  float64
 14  ball_control        183142 non-null  float64
 15  acceleration        183142 non-nul

In [30]:
# Averiguamos cuantas filas tienen al menos un valor nulo
df_player_attributes.isnull().T.any().T.sum()

2713

Estas filas con algún valor nulo representan sólo el 1,5% del Dataframe, para agilizar, decidimos eliminarlas todas para evitar los problemas que suele acarrear los nulos.

In [31]:
df_player_attributes.dropna(axis=0, how='any', inplace=True)

In [32]:
df_player_attributes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181265 entries, 0 to 183977
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   player_api_id       181265 non-null  int64  
 1   date                181265 non-null  object 
 2   overall_rating      181265 non-null  float64
 3   potential           181265 non-null  float64
 4   preferred_foot      181265 non-null  object 
 5   crossing            181265 non-null  float64
 6   finishing           181265 non-null  float64
 7   heading_accuracy    181265 non-null  float64
 8   short_passing       181265 non-null  float64
 9   volleys             181265 non-null  float64
 10  dribbling           181265 non-null  float64
 11  curve               181265 non-null  float64
 12  free_kick_accuracy  181265 non-null  float64
 13  long_passing        181265 non-null  float64
 14  ball_control        181265 non-null  float64
 15  acceleration        181265 non-null  fl

In [33]:
# Convertimos los tipos aquí también, ahora que nos hemos deshecho de los nulos, podemos convertir
# a los tipos enteros sin problema
df_player_attributes['date'] = pd.to_datetime(df_player_attributes['date'])
df_player_attributes['preferred_foot'] = df_player_attributes['preferred_foot'].astype('string')
df_player_attributes['overall_rating'] = df_player_attributes['overall_rating'].astype('uint8')
df_player_attributes['potential'] = df_player_attributes['potential'].astype('uint8')
df_player_attributes['crossing'] = df_player_attributes['crossing'].astype('uint8')
df_player_attributes['finishing'] = df_player_attributes['finishing'].astype('uint8')
df_player_attributes['heading_accuracy'] = df_player_attributes['heading_accuracy'].astype('uint8')
df_player_attributes['short_passing'] = df_player_attributes['short_passing'].astype('uint8')
df_player_attributes['volleys'] = df_player_attributes['volleys'].astype('uint8')
df_player_attributes['dribbling'] = df_player_attributes['dribbling'].astype('uint8')
df_player_attributes['curve'] = df_player_attributes['curve'].astype('uint8')
df_player_attributes['free_kick_accuracy'] = df_player_attributes['free_kick_accuracy'].astype('uint8')
df_player_attributes['long_passing'] = df_player_attributes['long_passing'].astype('uint8')
df_player_attributes['ball_control'] = df_player_attributes['ball_control'].astype('uint8')
df_player_attributes['acceleration'] = df_player_attributes['acceleration'].astype('uint8')
df_player_attributes['sprint_speed'] = df_player_attributes['sprint_speed'].astype('uint8')
df_player_attributes['agility'] = df_player_attributes['agility'].astype('uint8')
df_player_attributes['reactions'] = df_player_attributes['reactions'].astype('uint8')
df_player_attributes['balance'] = df_player_attributes['balance'].astype('uint8')
df_player_attributes['shot_power'] = df_player_attributes['shot_power'].astype('uint8')
df_player_attributes['jumping'] = df_player_attributes['jumping'].astype('uint8')
df_player_attributes['stamina'] = df_player_attributes['stamina'].astype('uint8')
df_player_attributes['strength'] = df_player_attributes['strength'].astype('uint8')
df_player_attributes['long_shots'] = df_player_attributes['long_shots'].astype('uint8')
df_player_attributes['aggression'] = df_player_attributes['aggression'].astype('uint8')
df_player_attributes['interceptions'] = df_player_attributes['interceptions'].astype('uint8')
df_player_attributes['positioning'] = df_player_attributes['positioning'].astype('uint8')
df_player_attributes['vision'] = df_player_attributes['vision'].astype('uint8')
df_player_attributes['penalties'] = df_player_attributes['penalties'].astype('uint8')
df_player_attributes['marking'] = df_player_attributes['marking'].astype('uint8')
df_player_attributes['standing_tackle'] = df_player_attributes['standing_tackle'].astype('uint8')
df_player_attributes['sliding_tackle'] = df_player_attributes['sliding_tackle'].astype('uint8')
df_player_attributes['gk_diving'] = df_player_attributes['gk_diving'].astype('uint8')
df_player_attributes['gk_handling'] = df_player_attributes['gk_handling'].astype('uint8')
df_player_attributes['gk_kicking'] = df_player_attributes['gk_kicking'].astype('uint8')
df_player_attributes['gk_positioning'] = df_player_attributes['gk_positioning'].astype('uint8')
df_player_attributes['gk_reflexes'] = df_player_attributes['gk_reflexes'].astype('uint8')


En este punto, <b>vamos a hacer un join entre las tablas de Jugadores y atributos de Jugadores.</b>

In [34]:
df_player_attr = pd.merge(df_player, df_player_attributes, on='player_api_id', how='inner')

In [35]:
# Vemos la info. del nuevo Dataframe
df_player_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181265 entries, 0 to 181264
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   player_api_id       181265 non-null  int64         
 1   player_name         181265 non-null  string        
 2   birthday            181265 non-null  datetime64[ns]
 3   height              181265 non-null  uint8         
 4   weight              181265 non-null  uint8         
 5   date                181265 non-null  datetime64[ns]
 6   overall_rating      181265 non-null  uint8         
 7   potential           181265 non-null  uint8         
 8   preferred_foot      181265 non-null  string        
 9   crossing            181265 non-null  uint8         
 10  finishing           181265 non-null  uint8         
 11  heading_accuracy    181265 non-null  uint8         
 12  short_passing       181265 non-null  uint8         
 13  volleys             181265 no

No lo habíamos comentado antes, pero aquí se aprecia perfectamente que la tabla/Dataframe de 
Player_Attributes contenía varios registros para cada jugador en los que se aprecia su evolución
en números como futbolista, el campo más descriptivo es <u>'overall_rating'</u>

Vamos a calcular una nueva columna que va a ser la edad del futbolista en la fecha en la que recogen los valores de esos atributos, para ello hay que restar 'date' y 'birthday', añadir ese resultado como nueva columna y deshacernos de las otras dos.

In [36]:
df_player_attr.insert(2, 'player_age', np.floor((df_player_attr['date'] - df_player_attr['birthday']).dt.days/365.2425).astype(int))

In [37]:
df_player_attr = df_player_attr.drop('birthday', axis=1).drop('date', axis=1)

In [38]:
df_player_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181265 entries, 0 to 181264
Data columns (total 41 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   player_api_id       181265 non-null  int64 
 1   player_name         181265 non-null  string
 2   player_age          181265 non-null  int32 
 3   height              181265 non-null  uint8 
 4   weight              181265 non-null  uint8 
 5   overall_rating      181265 non-null  uint8 
 6   potential           181265 non-null  uint8 
 7   preferred_foot      181265 non-null  string
 8   crossing            181265 non-null  uint8 
 9   finishing           181265 non-null  uint8 
 10  heading_accuracy    181265 non-null  uint8 
 11  short_passing       181265 non-null  uint8 
 12  volleys             181265 non-null  uint8 
 13  dribbling           181265 non-null  uint8 
 14  curve               181265 non-null  uint8 
 15  free_kick_accuracy  181265 non-null  uint8 
 16  lo

In [39]:
print ("El número de futbolistas de los que tenemos información es:",df_player_attr['player_api_id'].nunique())

El número de futbolistas de los que tenemos información es: 10582


<BR>

Vamos a realizar la unión de la tabla "Team" (<i>df_team</i>) con el dataframe de <i>df_match_league</i>, de manera que tengamos en los datos de los partidos los nombres de los equipos de ese partido, para ello, tenemos que hacer un inner join dos veces, una para el equipo local y otra para el visitante

In [40]:
df_match_league_team = pd.merge(left=df_match_league, right=df_team, left_on="home_team_api_id", right_on="team_api_id", how="inner")

In [41]:
df_match_league_team.drop("team_api_id", axis=1, inplace=True)
col_changed = df_match_league_team.pop('team')
df_match_league_team.insert(8, 'home_team_name', col_changed)

In [42]:
df_match_league_team = pd.merge(left=df_match_league_team, right=df_team, left_on="away_team_api_id", right_on="team_api_id", how="inner")

In [43]:
df_match_league_team.drop("team_api_id", axis=1, inplace=True)
col_changed = df_match_league_team.pop('team')
df_match_league_team.insert(10, 'away_team_name', col_changed)

In [44]:
df_match_league_team.info(max_cols=85)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 83 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   country           25979 non-null  string        
 1   league            25979 non-null  string        
 2   country_id        25979 non-null  uint32        
 3   season            25979 non-null  string        
 4   stage             25979 non-null  uint8         
 5   date              25979 non-null  datetime64[ns]
 6   match_api_id      25979 non-null  int64         
 7   home_team_api_id  25979 non-null  int64         
 8   home_team_name    25979 non-null  string        
 9   away_team_api_id  25979 non-null  int64         
 10  away_team_name    25979 non-null  string        
 11  home_team_goal    25979 non-null  uint8         
 12  away_team_goal    25979 non-null  uint8         
 13  home_player_X1    24158 non-null  float16       
 14  home_player_X2    2415

<b>Ahora vamos a extraer la información de los goles que se marcaron en la tabla <b>Match</b>, esta información viene en el campo <i>goal</i> en formato XML.</b> 
Vamos a comprobar cuantos campos tienen informada la columna <i>goal</i>, ordenando por temporada el resultado, también nos interesa el país de la liga en que se jugó ese partido.
Además, eliminaremos las filas nulas del Dataframe resultante

In [45]:
df_matches_goals_xmls = pd.DataFrame(df_match_league_team.loc[df_match_league_team.goal !='', \
                                                              ['match_api_id', 'country', 'goal', 'season']] \
                                     , index=df_match_league_team.index )

df_matches_goals_xmls.dropna(axis=0, inplace=True, how='any')        
df_matches_goals_xmls.sort_values(by=['season'])

,match_api_id,country,goal,season
1728,489042.0,England,<goal><value><comment>n</comment><stats><goals...,2008/2009
22486,530553.0,Spain,<goal><value><comment>n</comment><stats><goals...,2008/2009
22490,530334.0,Spain,<goal><value><comment>n</comment><stats><goals...,2008/2009
22494,530639.0,Spain,<goal><value><comment>n</comment><stats><goals...,2008/2009
9512,499444.0,Germany,<goal />,2008/2009
...,...,...,...,...
11816,2060287.0,Italy,<goal><value><comment>n</comment><stats><goals...,2015/2016
11808,2060597.0,Italy,<goal />,2015/2016
5399,1989781.0,France,<goal><value><comment>n</comment><stats><goals...,2015/2016
5366,1989813.0,France,<goal><value><comment>n</comment><stats><goals...,2015/2016


In [46]:
import time
start = time.time()
df_all_goals = pd.DataFrame()
countErrores,countOK = 0, 0
for idx in df_matches_goals_xmls.index:
    try:
        xml_goal = pd.read_xml(str(df_matches_goals_xmls.loc[idx, 'goal']))
        xml_goal.insert(0, 'match_api_id', df_matches_goals_xmls.loc[idx, 'match_api_id'])
        xml_goal.insert(1, 'country', df_matches_goals_xmls.loc[idx, 'country'])
        xml_goal.insert(2, 'season', df_matches_goals_xmls.loc[idx, 'season'])
        df_all_goals = pd.concat( [df_all_goals,xml_goal] )
    except ValueError:
        countErrores += 1
    else:    
        countOK += 1
df_all_goals.reset_index(inplace=True)        
print("errores:{}, NO Errores:{}".format(countErrores, countOK))   
print ('Time for merge with ALL completes Dataframes is {} sec'.format(time.time()-start))

errores:993, NO Errores:13224
Time for merge with ALL completes Dataframes is 56.56961917877197 sec


In [47]:
df_all_goals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  39980 non-null  int64  
 1   match_api_id           39980 non-null  float64
 2   country                39980 non-null  object 
 3   season                 39980 non-null  object 
 4   comment                39980 non-null  object 
 5   stats                  0 non-null      float64
 6   event_incident_typefk  39980 non-null  int64  
 7   elapsed                39980 non-null  int64  
 8   player2                17069 non-null  float64
 9   subtype                21551 non-null  object 
 10  player1                39863 non-null  float64
 11  sortorder              39980 non-null  int64  
 12  team                   39946 non-null  float64
 13  id                     39980 non-null  int64  
 14  n                      39980 non-null  int64  
 15  ty

Eliminamos todas las columnas que no nos interesen o que tengan muy difícil interpretación, también la columna <i>'type'</i> que no aporta nada ya que tiene un único valor: <i>'goal'</i>

In [48]:
df_all_goals = df_all_goals.drop('index', axis=1).drop('stats', axis=1).drop('coordinates', axis=1).drop('del', axis=1) \
    .drop('comment', axis=1).drop('event_incident_typefk', axis=1).drop('player2', axis=1).drop('n', axis=1) \
    .drop('sortorder', axis=1).drop('type', axis=1)

In [49]:
# Eliminamos todas las filas que tengan valor nulo ya sea en el id de jugador o en el del equipo, ya que afortunadamente
# en ambos casos, son pocos.

df_all_goals.dropna(axis=0, how='any', subset=['player1', 'team'], inplace=True)

#Sólo perdemos (39980-39863)/39980 = 0.2 % de los regsitros

In [50]:
# Comprobamos los valores y columnas con que nos quedamos
df_all_goals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39863 entries, 0 to 39979
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   match_api_id  39863 non-null  float64
 1   country       39863 non-null  object 
 2   season        39863 non-null  object 
 3   elapsed       39863 non-null  int64  
 4   subtype       21542 non-null  object 
 5   player1       39863 non-null  float64
 6   team          39863 non-null  float64
 7   id            39863 non-null  int64  
 8   goal_type     39863 non-null  object 
 9   elapsed_plus  1650 non-null   float64
dtypes: float64(4), int64(2), object(4)
memory usage: 3.3+ MB


Ya disponemos de cerca de 40K goles de las temporadas que van de 2008 a 2016. Aunque veremos que son datos relativos a goles, no todos son goles "estándar", por ejemplo, también hay goles en propia puerta .

In [51]:
# Convertimos las columnas de tipo objeto en 'string'
columns_type_object = list(df_all_goals.select_dtypes(include='object').columns)
for col in columns_type_object:
    df_all_goals[col] = df_all_goals[col].astype('string')
# Convertimos tres columnas concretas que son identificadores de float64 a int64    
df_all_goals['match_api_id'] = df_all_goals['match_api_id'].astype('int64')
df_all_goals['player1'] = df_all_goals['player1'].astype('int64')
df_all_goals['team'] = df_all_goals['team'].astype('int64')

Lo siguiente seria hacer un join de estos dos campos con los Dataframes de Jugadores y de equipos, básicamente enlazamos el <u>Match_api_id</u> del partido junto con la temporada (<u>season</u>) a el jugador (<u>player1</u>) que ha marcado gol y después hacemos el join con el id del equipo (<u>team</u>) para el que lo metió.

In [52]:
df_goals_player_teams = pd.merge(left=df_all_goals, right=df_player, left_on='player1', right_on='player_api_id', how='inner')
df_goals_player_teams = pd.merge(left=df_goals_player_teams, right=df_team, left_on='team', right_on='team_api_id', how='inner')

In [53]:
# Vamos a mostrar la información del nuevo dataframe df_goals_player_teams
df_goals_player_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39665 entries, 0 to 39664
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   match_api_id   39665 non-null  int64         
 1   country        39665 non-null  string        
 2   season         39665 non-null  string        
 3   elapsed        39665 non-null  int64         
 4   subtype        21442 non-null  string        
 5   player1        39665 non-null  int64         
 6   team_x         39665 non-null  int64         
 7   id             39665 non-null  int64         
 8   goal_type      39665 non-null  string        
 9   elapsed_plus   1640 non-null   float64       
 10  player_api_id  39665 non-null  int64         
 11  player_name    39665 non-null  string        
 12  birthday       39665 non-null  datetime64[ns]
 13  height         39665 non-null  uint8         
 14  weight         39665 non-null  uint8         
 15  team_api_id    3966

In [54]:
# Eliminamos algunas columnas y renombramos otras
df_goals_player_teams = df_goals_player_teams.drop('team_x', axis=1).drop('player1', axis=1)
df_goals_player_teams.rename(columns={'team_y':'team'}, inplace=True)

col_pos_changed = df_goals_player_teams.pop('team_api_id')
df_goals_player_teams.insert(5, 'team_api_id', col_pos_changed)
col_pos_changed = df_goals_player_teams.pop('team')
df_goals_player_teams.insert(6, 'team', col_pos_changed)


In [55]:
pd.set_option('display.max_columns', 500)
df_goals_player_teams.head(5)

,match_api_id,country,season,elapsed,subtype,team_api_id,team,id,goal_type,elapsed_plus,player_api_id,player_name,birthday,height,weight
0,489042,England,2008/2009,22,header,10261,Newcastle United,378998,n,NaN,37799,Obafemi Martins,1984-10-28,170,161
1,489208,England,2008/2009,77,shot,10261,Newcastle United,529875,n,NaN,37799,Obafemi Martins,1984-10-28,170,161
2,489166,England,2008/2009,87,shot,10261,Newcastle United,488436,n,NaN,37799,Obafemi Martins,1984-10-28,170,161
3,489397,England,2008/2009,71,shot,10261,Newcastle United,679225,n,NaN,37799,Obafemi Martins,1984-10-28,170,161
4,489146,England,2008/2009,60,distance,10261,Newcastle United,473071,n,NaN,37799,Obafemi Martins,1984-10-28,170,161


In [56]:
df_goals_player_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39665 entries, 0 to 39664
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   match_api_id   39665 non-null  int64         
 1   country        39665 non-null  string        
 2   season         39665 non-null  string        
 3   elapsed        39665 non-null  int64         
 4   subtype        21442 non-null  string        
 5   team_api_id    39665 non-null  int64         
 6   team           39665 non-null  string        
 7   id             39665 non-null  int64         
 8   goal_type      39665 non-null  string        
 9   elapsed_plus   1640 non-null   float64       
 10  player_api_id  39665 non-null  int64         
 11  player_name    39665 non-null  string        
 12  birthday       39665 non-null  datetime64[ns]
 13  height         39665 non-null  uint8         
 14  weight         39665 non-null  uint8         
dtypes: datetime64[ns](1

In [57]:
# Vamos a obtener todos los valores posibles de goal_type y estudiar cada caso
df_all_goals['goal_type'].unique()

<StringArray>
['n', 'p', 'dg', 'o', 'npm', 'psm', 'rp']
Length: 7, dtype: string

In [58]:
df_goals_player_teams.loc[df_goals_player_teams['goal_type']=='psm']

,match_api_id,country,season,elapsed,subtype,team_api_id,team,id,goal_type,elapsed_plus,player_api_id,player_name,birthday,height,weight
14980,489229,England,2008/2009,45,<NA>,8462,Portsmouth,538898,psm,2.0,30348,Jermain Defoe,1982-10-07,170,154


El pequeño estudio del significado de los valores de las columnas 'goal_type' da este resultado: 
- n --> gol normal
- p --> gol de penalti
- dg --> otro incidente de juego, desconocido, pero no gol
- o --> gol en propia puerta
- npm --> penalti fallado
- psm --> sin apenas datos, sólo un caso, penalti fallado
- rp --> gol de rechace de penalti, o de repetición de penalti¿¿??

Con lo que vamos a considerar goles a contabilizar: 'n', 'p' y 'rp'

También se va a convertir el campo season de la forma 2008/2009 a la forma 2008 para hacer join con este campo

In [59]:
def extract_substring(value):
    return value[0:4]

df_goals_player_teams['season'] = df_goals_player_teams['season'].apply(extract_substring)
df_goals_player_teams['season'] = df_goals_player_teams['season'].astype('uint16')

In [60]:
df_goals_player_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39665 entries, 0 to 39664
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   match_api_id   39665 non-null  int64         
 1   country        39665 non-null  string        
 2   season         39665 non-null  uint16        
 3   elapsed        39665 non-null  int64         
 4   subtype        21442 non-null  string        
 5   team_api_id    39665 non-null  int64         
 6   team           39665 non-null  string        
 7   id             39665 non-null  int64         
 8   goal_type      39665 non-null  string        
 9   elapsed_plus   1640 non-null   float64       
 10  player_api_id  39665 non-null  int64         
 11  player_name    39665 non-null  string        
 12  birthday       39665 non-null  datetime64[ns]
 13  height         39665 non-null  uint8         
 14  weight         39665 non-null  uint8         
dtypes: datetime64[ns](1

<br><br>

Ahora <b>lo mismo que se ha hecho para obtener un Dataframe de los goles por jugadores y equipos, se va a hacer con las tarjetas recibidas</b> para tener otro indicador que afecte más a los jugadores de posiciones defensivas.

In [61]:
df_matches_cards_xmls = pd.DataFrame(df_match_league_team.loc[df_match_league_team.card !='', \
                                                              ['match_api_id', 'country', 'card', 'season']] \
                                     , index=df_match_league_team.index )

In [62]:
df_matches_cards_xmls.dropna(axis=0, inplace=True, how='any')   
df_matches_cards_xmls.sort_values(by=['season'])

,match_api_id,country,card,season
1728,489042.0,England,<card><value><comment>y</comment><stats><ycard...,2008/2009
22486,530553.0,Spain,<card><value><comment>y</comment><stats><ycard...,2008/2009
22490,530334.0,Spain,<card><value><comment>y</comment><stats><ycard...,2008/2009
22494,530639.0,Spain,<card><value><comment>y</comment><stats><ycard...,2008/2009
9512,499444.0,Germany,<card><value><comment>y</comment><stats><ycard...,2008/2009
...,...,...,...,...
11816,2060287.0,Italy,<card><value><comment>y</comment><stats><ycard...,2015/2016
11808,2060597.0,Italy,<card><value><comment>y</comment><stats><ycard...,2015/2016
5399,1989781.0,France,<card><value><comment>y</comment><stats><ycard...,2015/2016
5366,1989813.0,France,<card><value><comment>y</comment><stats><ycard...,2015/2016


In [63]:
import time
start = time.time()
df_all_cards = pd.DataFrame()
countParserErrors, countParserOK = 0, 0
for idx in df_matches_cards_xmls.index:
    try:
        xml_card = pd.read_xml(str(df_matches_cards_xmls.loc[idx, 'card']))
        xml_card.insert(0, 'match_api_id', df_matches_cards_xmls.loc[idx, 'match_api_id'])
        xml_card.insert(1, 'country', df_matches_cards_xmls.loc[idx, 'country'])
        xml_card.insert(2, 'season', df_matches_cards_xmls.loc[idx, 'season'])
        df_all_cards = pd.concat( [df_all_cards,xml_card] )
    except ValueError:
        countParserErrors += 1
    else:    
        countParserOK += 1
df_all_cards.reset_index(inplace=True)   
print ('Tiempo para recorrer todos los campos card {} segundos.'.format(time.time()-start))
print("Número de errores de parseo:{}, Número de parseos correctos:{}".format(countParserErrors, countParserOK))

Tiempo para recorrer todos los campos card 66.2469871044159 segundos.
Número de errores de parseo:441, Número de parseos correctos:13776


In [64]:
df_all_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62114 entries, 0 to 62113
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  62114 non-null  int64  
 1   match_api_id           62114 non-null  float64
 2   country                62114 non-null  object 
 3   season                 62114 non-null  object 
 4   comment                62114 non-null  object 
 5   stats                  0 non-null      float64
 6   event_incident_typefk  62114 non-null  int64  
 7   elapsed                62114 non-null  int64  
 8   card_type              61509 non-null  object 
 9   subtype                18864 non-null  object 
 10  player1                61794 non-null  object 
 11  sortorder              62114 non-null  int64  
 12  team                   61794 non-null  float64
 13  n                      62114 non-null  int64  
 14  type                   62114 non-null  object 
 15  id

Eliminamos todas las columnas que no nos interesen, tengan muy difícil interpretación, o no sean necesarias, por ejemplo la columna <i>'type'</i> que no aporta nada ya que tiene un único valor: <i>'goal'</i>

In [65]:
df_all_cards = df_all_cards.drop('index', axis=1).drop('stats', axis=1).drop('subtype', axis=1).drop('del', axis=1) \
    .drop('comment', axis=1).drop('event_incident_typefk', axis=1).drop('n', axis=1) \
    .drop('sortorder', axis=1).drop('goal_type', axis=1).drop('type', axis=1)

In [66]:
# Al igual que con los goles, eliminamos todas las filas que tengan valor nulo ya sea en el id de jugador, el del equipo o 
# en este caso, el tipo de tarjeta,también aquí son pocos: no llega a un 1% de los registros los que se borran.
df_all_cards.dropna(axis=0, how='any', subset=['player1', 'team', 'card_type'], inplace=True)
df_all_cards.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61509 entries, 0 to 62113
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   match_api_id  61509 non-null  float64
 1   country       61509 non-null  object 
 2   season        61509 non-null  object 
 3   elapsed       61509 non-null  int64  
 4   card_type     61509 non-null  object 
 5   player1       61509 non-null  object 
 6   team          61509 non-null  float64
 7   id            61509 non-null  int64  
 8   elapsed_plus  3422 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 4.7+ MB


Así que disponemos de 61K datos de tarjetas de las temporadas que van de 2008 a 2016

In [67]:
# Hemos comprobado que en player1 no todos son números, hay tres registros cuyo valor es "Unknown player"
df_all_cards.drop(index = df_all_cards.loc[df_all_cards.player1=='Unknown player'].index, inplace=True)
df_all_cards['player1'] = df_all_cards['player1'].astype('float64')
df_all_cards['player1'] = df_all_cards['player1'].astype('int64')    

In [68]:
# Convertimos las columnas de tipo objeto en 'string'
columns_type_object = list(df_all_cards.select_dtypes(include='object').columns)
for col in columns_type_object:
    df_all_cards[col] = df_all_cards[col].astype('string')
# Convertimos dos columnas concretas a int64    
df_all_cards['match_api_id'] = df_all_cards['match_api_id'].astype('int64')
df_all_cards['team'] = df_all_cards['team'].astype('int64')

In [69]:
df_all_cards.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61506 entries, 0 to 62113
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   match_api_id  61506 non-null  int64  
 1   country       61506 non-null  string 
 2   season        61506 non-null  string 
 3   elapsed       61506 non-null  int64  
 4   card_type     61506 non-null  string 
 5   player1       61506 non-null  int64  
 6   team          61506 non-null  int64  
 7   id            61506 non-null  int64  
 8   elapsed_plus  3422 non-null   float64
dtypes: float64(1), int64(5), string(3)
memory usage: 4.7 MB


Al igual que con el dataframe <b>df_all_goals</b> lo siguiente con el Dataframe <b>df_all_cards</b> es un join con los Dataframes de Jugadores y de equipos, básicamente enlazamos el <i>match_api_id</i> del partido junto con la temporada (<i>season</i>) a el jugador (<i>player1</i>) que ha recibido la tarjeta y después hacemos el join con el id del equipo (<i>team</i>) en el que jugaba entonces.
De esta forma creamos un nuevo Dataframe: <b>df_cards_players_teams</b>

In [70]:
df_cards_players_teams = pd.merge(left=df_all_cards, right=df_player, left_on='player1', right_on='player_api_id', how='inner')
df_cards_players_teams = pd.merge(left=df_cards_players_teams, right=df_team, left_on='team', right_on='team_api_id', how='inner')

In [71]:
# Vamos a mostrar la información del nuevo dataframe df_cards_players_teams
df_cards_players_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61095 entries, 0 to 61094
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   match_api_id   61095 non-null  int64         
 1   country        61095 non-null  string        
 2   season         61095 non-null  string        
 3   elapsed        61095 non-null  int64         
 4   card_type      61095 non-null  string        
 5   player1        61095 non-null  int64         
 6   team_x         61095 non-null  int64         
 7   id             61095 non-null  int64         
 8   elapsed_plus   3392 non-null   float64       
 9   player_api_id  61095 non-null  int64         
 10  player_name    61095 non-null  string        
 11  birthday       61095 non-null  datetime64[ns]
 12  height         61095 non-null  uint8         
 13  weight         61095 non-null  uint8         
 14  team_api_id    61095 non-null  int64         
 15  team_y         6109

In [72]:
# Al igual que lo que se hizo con el otro Dataframe citado antes <b>df_goals_players_teams</b>, eliminamos algunas columnas y renombramos otras
df_cards_players_teams = df_cards_players_teams.drop('team_x', axis=1).drop('player1', axis=1)
df_cards_players_teams.rename(columns={'team_y':'team'}, inplace=True)

In [73]:
#Reordenamos algunas columnas y vemos algunas filas
col_pos_changed = df_cards_players_teams.pop('id')
df_cards_players_teams.insert(0, 'id', col_pos_changed)
col_pos_changed = df_cards_players_teams.pop('team_api_id')
df_cards_players_teams.insert(7, 'team_api_id', col_pos_changed)
col_pos_changed = df_cards_players_teams.pop('team')
df_cards_players_teams.insert(8, 'team', col_pos_changed)

df_cards_players_teams.head()

,id,match_api_id,country,season,elapsed,card_type,elapsed_plus,team_api_id,team,player_api_id,player_name,birthday,height,weight
0,379481,489042,England,2008/2009,78,y,NaN,10260,Manchester United,24157,Fraizer Campbell,1987-09-13,172,183
1,379503,489042,England,2008/2009,82,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170
2,741683,658603,England,2009/2010,62,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170
3,1272985,839936,England,2010/2011,71,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170
4,456957,489125,England,2008/2009,85,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170


In [74]:
# Obtenemos los valores posibles de card_type: 
df_all_cards['card_type'].unique()
# Vemos que son 
# y --> amarilla
# y2 --> segunda amarilla
# r --> roja

<StringArray>
['y', 'y2', 'r']
Length: 3, dtype: string

In [75]:
df_cards_players_teams.loc[(df_cards_players_teams.card_type == 'y2') | (df_cards_players_teams.card_type == 'r')] \
                           [['card_type', 'elapsed', 'elapsed_plus', 'match_api_id', 'player_name', 'team']]\
                        .sort_values(by=['match_api_id', 'elapsed', 'elapsed_plus'])

# Comprobamos que no se registra la tarjeta roja cuando hay una segunda amarilla

,card_type,elapsed,elapsed_plus,match_api_id,player_name,team
17970,y2,38,NaN,489055,Mark Noble,West Ham United
11759,r,36,NaN,489068,Amdy Faye,Stoke City
311,y2,90,1.0,489072,Nemanja Vidic,Manchester United
1919,r,90,6.0,489074,Danny Guthrie,Newcastle United
10165,r,77,NaN,489075,John Terry,Chelsea
...,...,...,...,...,...,...
60034,r,13,NaN,2060640,Mirko Gori,Frosinone
9571,r,34,NaN,2060641,Michel Morganella,Palermo
56458,r,35,NaN,2060641,Pawel Wszolek,Hellas Verona
13368,y2,60,NaN,2060642,Jeison Murillo,Inter


In [76]:
df_cards_players_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61095 entries, 0 to 61094
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             61095 non-null  int64         
 1   match_api_id   61095 non-null  int64         
 2   country        61095 non-null  string        
 3   season         61095 non-null  string        
 4   elapsed        61095 non-null  int64         
 5   card_type      61095 non-null  string        
 6   elapsed_plus   3392 non-null   float64       
 7   team_api_id    61095 non-null  int64         
 8   team           61095 non-null  string        
 9   player_api_id  61095 non-null  int64         
 10  player_name    61095 non-null  string        
 11  birthday       61095 non-null  datetime64[ns]
 12  height         61095 non-null  uint8         
 13  weight         61095 non-null  uint8         
dtypes: datetime64[ns](1), float64(1), int64(5), string(5), uint8(2)
memory

Aquí también hay que convertir el campo season de la forma 2008/2009 a la forma 2008 para hacer join con este campo

In [77]:
def extract_substring(value):
    return value[0:4]

df_cards_players_teams['season'] = df_cards_players_teams['season'].apply(extract_substring)
df_cards_players_teams['season'] = df_cards_players_teams['season'].astype('uint16')

In [78]:
df_cards_players_teams.head(5)

,id,match_api_id,country,season,elapsed,card_type,elapsed_plus,team_api_id,team,player_api_id,player_name,birthday,height,weight
0,379481,489042,England,2008,78,y,NaN,10260,Manchester United,24157,Fraizer Campbell,1987-09-13,172,183
1,379503,489042,England,2008,82,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170
2,741683,658603,England,2009,62,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170
3,1272985,839936,England,2010,71,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170
4,456957,489125,England,2008,85,y,NaN,10260,Manchester United,30362,Wes Brown,1979-10-13,185,170


<br>

<br>

## 2º FUENTE DE DATOS: FOOTBALL-TRANSFERS-DATA
Se encuentra en la URL: https://github.com/d2ski/football-transfers-data/ en el repositorio de Dmitrii Antipov.

Consiste en un único archivo CSV con datos de traspasos de jugadores de un club a otro, fecha en la que se produjo, y varios datos del jugador, especialmente el valor de mercado que es el que más nos interesa.
Antes de cargarlos hemos hecho algún tratamiento directamente en el archivo que ha consistido en redondear las cantidades monetarias, ya que todas tenían decimales innecesarios, por ejemplo: 2099999.99999 pasa a ser 2100000, 24.0 pasa a ser 24, y así las demás. 

El caso es que el INNER JOIN entre  ambos Dataframes(<i>'df_player_attr'</i> y <i>'df_transfers'</i>) utilizando dos columnas ['player_name','player_age'],no ofrece fiablilidad, ya que puede haber futbolistas con exactamente el mismo nombre y que en dos momentos distintos del tiempo "coinciden en edad", como consecuencia, se ve que hace falta utilizar más criterios para que el merge sea más fiable.

Se busca otra solución, tras realizar un ajuste de los campos de cantidades del propio archivo CSV, de los traspasos (<i>transfers</i>) de esta 2ª Fuente de Datos, que son todos enteros y no necesitan ser reales; se decide, a continuación, importar el CSV como una tabla más en la BD SQLite, se importa como <i>transfers_cleaned</i>. 
De este modo, se resta dificultad a la compleja tarea de relacionar esta tabla <i>transfers_cleaned</i> con las demás, el uso del SQL nos va a permitir relacionar en una consulta: Tabla de Jugadores de la 1ª Fuente, Tabla de Equipos de la 1ª Fuente y Tabla de traspasos de la 2ª Fuente. Y para ello nos vamos a apoyar en la tabla de Partidos pues parece un modo más fiable que el probado unas celdas más arriba. Lo que  se quiere hacer es comprobar que futbolistas han jugado en que partidos de que temporadas y defendiendo los colores de que equipos, de esa manera se va a relacionar:
 - Los nombres exactos de futbolistas en las tablas de Jugadores y de Traspasos.
 - La edad que cumplirá un futbolista en el año en que juega un partido (cualquiera) con su nuevo equipo y la que aparece en la tabla de traspasos en el traspaso a ese equipo ese año.
 - Por supuesto, que el futbolista este en la alineación de ese partido de ese equipo.
 - Que dispongamos de una cantidad de precio de mercado de ese futbolista (no la cantidad que se pago en el traspaso)

El resultado es el siguiente:

In [ ]:
'''
import time
start = time.time()

cnx.execute('PRAGMA case_sensitive_like = true')
df_mkt_player_team_matches = pd.read_sql_query(" \
SELECT DISTINCT pl.player_name, \
    pl.player_api_id, \
    tr.player_nation, \
    tr.season, \
    t.team_long_name, \
    t.team_api_id, \
    cast(tr.season AS integer) - cast(strftime('%Y', pl.birthday) AS integer) AS 'season_player_age', \
    tr.market_val_amnt, \
    tr.player_pos \
FROM \
    Player pl, Match m , Team t, transfers_cleaned tr \
WHERE \
    (pl.player_name LIKE '%'||tr.player_name||'%' OR tr.player_name LIKE '%'||pl.player_name||'%') AND \
    (t.team_long_name LIKE '%'||tr.team_name||'%' OR tr.team_name LIKE '%'||t.team_long_name||'%') AND \
    ROUND((unixepoch(m.date) - unixepoch(pl.birthday))/86400/365.2425 - 0.5) = tr.player_age AND \
    tr.market_val_amnt <> '' AND tr.market_val_amnt>0  \
    AND tr.dir = 'in' \
    AND (( \
    (m.home_player_1=pl.player_api_id OR m.home_player_2=pl.player_api_id OR m.home_player_3=pl.player_api_id \
     OR m.home_player_4=pl.player_api_id OR m.home_player_5=pl.player_api_id OR m.home_player_6=pl.player_api_id \
     OR m.home_player_7=pl.player_api_id OR m.home_player_8=pl.player_api_id OR m.home_player_9=pl.player_api_id \
     OR m.home_player_10=pl.player_api_id OR m.home_player_11=pl.player_api_id) \
    AND m.home_team_api_id=t.team_api_id ) \
    OR \
    ((m.away_player_1=pl.player_api_id OR m.away_player_2=pl.player_api_id OR m.away_player_3=pl.player_api_id \
        OR m.away_player_4=pl.player_api_id OR m.away_player_5=pl.player_api_id OR m.away_player_6=pl.player_api_id \
        OR m.away_player_7=pl.player_api_id OR m.away_player_8=pl.player_api_id OR m.away_player_9=pl.player_api_id \
        OR m.away_player_10=pl.player_api_id OR m.away_player_11=pl.player_api_id) \
     AND m.away_team_api_id=t.team_api_id \
    )) \
ORDER BY pl.player_name,tr.season, market_val_amnt DESC" , cnx )

print ('Tiempo que ha necesitado este SELECT con 4 tablas y 2 LIKEs  {} min.'.format((time.time()-start)/60))
'''
# !!!! Esta consulta puede durar entre 10 y 15 minutos !!!!

In [81]:
# Si queremos podemos escribir un archivo CSV que nos permita ver visualmente el resultado.
#df_mkt_player_team_matches.to_csv('df_mkt_player_team_matches.csv', encoding='utf-8', header=True, index=False)

<b>Si ya teníamos el Dataframe guardado en disco,</b> en vez de ejecutar la costosa query, realizamos la carga del Dataframe leyendo el CSV . 

In [82]:
df_mkt_player_team_matches = pd.read_csv('df_mkt_player_team_matches.csv', header=0)

In [83]:
df_mkt_player_team_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3420 entries, 0 to 3419
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   player_name        3420 non-null   object
 1   player_api_id      3420 non-null   int64 
 2   player_nation      3420 non-null   object
 3   season             3420 non-null   int64 
 4   team_long_name     3420 non-null   object
 5   team_api_id        3420 non-null   int64 
 6   season_player_age  3420 non-null   int64 
 7   market_val_amnt    3420 non-null   int64 
 8   player_pos         3420 non-null   object
dtypes: int64(5), object(4)
memory usage: 240.6+ KB


In [84]:
# Nuestro dataset se ha reducido a unas 3400 filas de traspasos, pero en realidad el número de futbolistas que tenemos es
print("Número total de jugadores: ",df_mkt_player_team_matches['player_api_id'].nunique())
df_mkt_player_team_matches.head(10)

Número total de jugadores:  2068


,player_name,player_api_id,player_nation,season,team_long_name,team_api_id,season_player_age,market_val_amnt,player_pos
0,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB
1,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM
2,Aaron Lennon,30895,England,2014,Everton,8668,27,1100000,RW
3,Aaron Lennon,30895,England,2015,Everton,8668,28,1100000,RW
4,Aaron Meijers,101042,Netherlands,2012,ADO Den Haag,10217,25,325000,LB
5,Aaron Ramsey,75489,Wales,2010,Arsenal,9825,20,10000000,CM
6,Abdelaziz Barrada,167027,Morocco,2011,Getafe CF,8305,22,200000,AM
7,Abdelhamid El Kaoutari,128456,Morocco,2015,Palermo,8540,25,400000,CB
8,Abdou Diallo,563215,Senegal,2014,AS Monaco,9829,18,25000000,LB
9,Abdoul Ba,564712,Mauritania,2014,RC Lens,8588,20,250000,CB


En este punto tenemos:
- por un lado tenemos el Dataframe de partidos <b><i>'df_match_league'</i></b> con casi 26k registros y 82 columnas : reúne información de ligas y partidos.
- el Dataframe con toda la información sobre goles que hemos podido reunir <b><i>'df_goals_players_teams'</i></b> con casi 40K registros y 15 columnas.
- el Dataframe con toda la información sobre tarjetas recibidas que hemos podido reunir <b><i>'df_cards_players_teams'</i></b> con 61K registros y 14 columnas.
- el Dataframe en el que hemos realizado una compleja unión usando las variables que guardan la alineación de cada partido de ambos equipos con las tablas 'Match', 'Player', 'Team' y la tabla de la segunda fuente de datos 'Transfers_cleaned', creando el DF que hemos llamado <b><i>'df_mkt_player_team_matches'</i></b>, han quedado 3420 registros con 9 columnas en las que encontramos unos 2K futbolistas únicos.
- No olvidar el Dataframe <b><i>'df_player_attr'</i></b> que recoge la evolución de los atributos futbolísticos de los jugadores en distintos momentos del tiempo con 181k registros y 41 columnas.
<br><br>

Ahora vamos a hacer otro merge de 'df_mkt_player_team_matches' con 'df_player_attr' en base al player_api_id y la edad del futbolista en la temporada de referencia 

In [85]:
df_mkt_player_team_matches_attributes = pd.merge(df_mkt_player_team_matches, df_player_attr, left_on=['player_api_id','season_player_age'], 
                                        right_on=['player_api_id','player_age'], how='inner')

In [86]:
df_mkt_player_team_matches_attributes.head(10)

,player_name_x,player_api_id,player_nation,season,team_long_name,team_api_id,season_player_age,market_val_amnt,player_pos,player_name_y,player_age,height,weight,overall_rating,potential,preferred_foot,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,Aaron Cresswell,25,170,146,73,77,left,79,51,57,70,29,71,68,69,68,70,79,78,78,67,90,71,84,79,56,58,67,66,58,65,59,76,75,78,14,7,9,9,12
1,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,Aaron Cresswell,25,170,146,74,78,left,79,51,57,70,29,71,68,69,68,70,80,78,78,67,90,71,84,79,56,58,67,66,58,65,59,76,75,78,14,7,9,9,12
2,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,Aaron Cresswell,25,170,146,73,77,left,79,51,57,70,29,67,68,69,68,68,79,78,78,67,90,71,84,79,56,58,67,66,58,65,59,76,75,78,14,7,9,9,12
3,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,Aaron Cresswell,25,170,146,71,75,left,78,50,56,69,28,66,67,68,67,67,79,82,79,71,90,70,84,79,50,56,66,65,57,64,58,73,72,72,13,6,8,8,11
4,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,Aaron Hunt,28,182,161,77,77,left,77,72,65,80,77,78,78,76,70,78,77,78,78,79,73,76,69,77,65,75,57,54,77,76,82,36,37,35,14,11,6,15,14
5,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,Aaron Hunt,28,182,161,77,78,left,77,72,65,80,77,78,78,76,70,78,77,78,78,79,73,76,69,77,65,75,57,54,77,76,82,36,37,35,14,11,6,15,14
6,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,Aaron Hunt,28,182,161,78,78,left,77,72,65,80,77,78,78,76,70,78,77,78,78,80,73,76,69,77,65,75,57,54,78,80,82,36,37,35,14,11,6,15,14
7,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,Aaron Hunt,28,182,161,78,78,left,77,72,65,80,77,78,78,76,70,78,77,78,78,80,73,76,69,77,66,75,57,54,78,80,82,36,37,35,14,11,6,15,14
8,Aaron Lennon,30895,England,2014,Everton,8668,27,1100000,RW,Aaron Lennon,27,165,139,78,78,right,78,65,27,77,67,83,62,54,53,80,92,89,93,77,92,61,71,75,52,56,56,36,73,75,61,29,33,34,13,6,6,15,10
9,Aaron Lennon,30895,England,2014,Everton,8668,27,1100000,RW,Aaron Lennon,27,165,139,78,78,right,78,65,27,77,67,83,62,54,53,80,92,89,93,77,92,61,71,75,52,56,56,36,73,75,61,25,23,25,13,6,6,15,10


In [87]:
# Eliminamos algunas columna redundantes y renombramo otra
df_mkt_player_team_matches_attributes.drop('player_name_y', axis=1, inplace=True)
df_mkt_player_team_matches_attributes.drop('player_age', axis=1, inplace=True)
df_mkt_player_team_matches_attributes.rename(columns={'player_name_x':'player_name'}, inplace=True)

In [88]:
df_mkt_player_team_matches_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13595 entries, 0 to 13594
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   player_name         13595 non-null  object
 1   player_api_id       13595 non-null  int64 
 2   player_nation       13595 non-null  object
 3   season              13595 non-null  int64 
 4   team_long_name      13595 non-null  object
 5   team_api_id         13595 non-null  int64 
 6   season_player_age   13595 non-null  int64 
 7   market_val_amnt     13595 non-null  int64 
 8   player_pos          13595 non-null  object
 9   height              13595 non-null  uint8 
 10  weight              13595 non-null  uint8 
 11  overall_rating      13595 non-null  uint8 
 12  potential           13595 non-null  uint8 
 13  preferred_foot      13595 non-null  string
 14  crossing            13595 non-null  uint8 
 15  finishing           13595 non-null  uint8 
 16  heading_accuracy    13

In [89]:
# Eliminamos duplicados quedando cerca de 11K registros
df_mkt_player_team_matches_attributes.drop_duplicates(inplace=True)
df_mkt_player_team_matches_attributes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11007 entries, 0 to 13594
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   player_name         11007 non-null  object
 1   player_api_id       11007 non-null  int64 
 2   player_nation       11007 non-null  object
 3   season              11007 non-null  int64 
 4   team_long_name      11007 non-null  object
 5   team_api_id         11007 non-null  int64 
 6   season_player_age   11007 non-null  int64 
 7   market_val_amnt     11007 non-null  int64 
 8   player_pos          11007 non-null  object
 9   height              11007 non-null  uint8 
 10  weight              11007 non-null  uint8 
 11  overall_rating      11007 non-null  uint8 
 12  potential           11007 non-null  uint8 
 13  preferred_foot      11007 non-null  string
 14  crossing            11007 non-null  uint8 
 15  finishing           11007 non-null  uint8 
 16  heading_accuracy    11007 n

In [90]:
columns_type_object = list(df_mkt_player_team_matches_attributes.select_dtypes(include='object').columns)
for col in columns_type_object:
    df_mkt_player_team_matches_attributes[col] = df_mkt_player_team_matches_attributes[col].astype('string')
# Convertimos dos columnas concretas a int64    
df_mkt_player_team_matches_attributes['season'] = df_mkt_player_team_matches_attributes['season'].astype('uint16')
df_mkt_player_team_matches_attributes['season_player_age'] = df_mkt_player_team_matches_attributes['season_player_age'].astype('uint8')

In [91]:
print("Número total de jugadores en este nuevo Dataframe: ",df_mkt_player_team_matches_attributes['player_api_id'].nunique())
df_mkt_player_team_matches_attributes.head(10)

Número total de jugadores en este nuevo Dataframe:  2024


,player_name,player_api_id,player_nation,season,team_long_name,team_api_id,season_player_age,market_val_amnt,player_pos,height,weight,overall_rating,potential,preferred_foot,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,170,146,73,77,left,79,51,57,70,29,71,68,69,68,70,79,78,78,67,90,71,84,79,56,58,67,66,58,65,59,76,75,78,14,7,9,9,12
1,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,170,146,74,78,left,79,51,57,70,29,71,68,69,68,70,80,78,78,67,90,71,84,79,56,58,67,66,58,65,59,76,75,78,14,7,9,9,12
2,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,170,146,73,77,left,79,51,57,70,29,67,68,69,68,68,79,78,78,67,90,71,84,79,56,58,67,66,58,65,59,76,75,78,14,7,9,9,12
3,Aaron Cresswell,155782,England,2014,West Ham United,8654,25,5000000,LB,170,146,71,75,left,78,50,56,69,28,66,67,68,67,67,79,82,79,71,90,70,84,79,50,56,66,65,57,64,58,73,72,72,13,6,8,8,11
4,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,182,161,77,77,left,77,72,65,80,77,78,78,76,70,78,77,78,78,79,73,76,69,77,65,75,57,54,77,76,82,36,37,35,14,11,6,15,14
5,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,182,161,77,78,left,77,72,65,80,77,78,78,76,70,78,77,78,78,79,73,76,69,77,65,75,57,54,77,76,82,36,37,35,14,11,6,15,14
6,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,182,161,78,78,left,77,72,65,80,77,78,78,76,70,78,77,78,78,80,73,76,69,77,65,75,57,54,78,80,82,36,37,35,14,11,6,15,14
7,Aaron Hunt,27316,Germany,2014,VfL Wolfsburg,8721,28,250000,AM,182,161,78,78,left,77,72,65,80,77,78,78,76,70,78,77,78,78,80,73,76,69,77,66,75,57,54,78,80,82,36,37,35,14,11,6,15,14
8,Aaron Lennon,30895,England,2014,Everton,8668,27,1100000,RW,165,139,78,78,right,78,65,27,77,67,83,62,54,53,80,92,89,93,77,92,61,71,75,52,56,56,36,73,75,61,29,33,34,13,6,6,15,10
9,Aaron Lennon,30895,England,2014,Everton,8668,27,1100000,RW,165,139,78,78,right,78,65,27,77,67,83,62,54,53,80,92,89,93,77,92,61,71,75,52,56,56,36,73,75,61,25,23,25,13,6,6,15,10


Vamos a listar las distintas demarcaciones que registra Transfermarket para los futbolistas en la columna 'player_pos'

Clasificamos las demarcaciones que existen en Transfermarket en las demarcaciones más simples y conocidas de Porteros, defensas, Centrocampistas, Delanteros o en todo caso, Centrocampistas defensivos o Centrocampistas opensivos:

GK -> Goalkeeper, PORTERO

defence -> DEFENSA 
RB -> Right Back, DEFENSA 
CB -> Centre Back, DEFENSA 
LB -> Left Back, DEFENSA 

DM -> Defensive Midfield, DEFENSA/CENTROCAMPISTA 

LM -> Left Midfield, CENTROCAMPISTA 
RM -> Right Midfield, CENTROCAMPISTA 
CM -> Central Midfield, CENTROCAMPISTA 
midfield -> CENTROCAMPISTA  

AM -> Attacking Midfield, CENTROCAMPISTA/DELANTERO 
LW -> Left Winger, CENTROCAMPISTA/DELANTERO 
RW -> Right Winger, CENTROCAMPISTA/DELANTERO 

CF -> Centre Forward , DELANTERO 
SS -> Second Striker, DELANTERO 
attack -> DELANTERO 

In [95]:
df_mkt_player_team_matches_attributes['player_pos'].unique()

<StringArray>
[      'LB',       'AM',       'RW',       'CM',       'CB',       'GK',
       'DM',       'CF',       'SS',       'LW',       'RB',       'RM',
       'LM',   'attack', 'midfield']
Length: 15, dtype: string

Ahora <b>vamos a hacer otro merge</b> de 'df_mkt_player_team_matches' con 'df_goals_players_teams' en base al player_api_id y la  temporada de referencia.

In [97]:
df_mkt_player_team_matches_goals = pd.merge(df_mkt_player_team_matches, df_goals_player_teams, \
        on=['player_api_id','season'], how='inner')

In [98]:
df_mkt_player_team_matches_goals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5561 entries, 0 to 5560
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   player_name_x      5561 non-null   object        
 1   player_api_id      5561 non-null   int64         
 2   player_nation      5561 non-null   object        
 3   season             5561 non-null   int64         
 4   team_long_name     5561 non-null   object        
 5   team_api_id_x      5561 non-null   int64         
 6   season_player_age  5561 non-null   int64         
 7   market_val_amnt    5561 non-null   int64         
 8   player_pos         5561 non-null   object        
 9   match_api_id       5561 non-null   int64         
 10  country            5561 non-null   string        
 11  elapsed            5561 non-null   int64         
 12  subtype            3211 non-null   string        
 13  team_api_id_y      5561 non-null   int64         
 14  team    

In [99]:
# Eliminamos algunas columnas redundantes y renombramos otras
df_mkt_player_team_matches_goals.drop('player_name_y', axis=1, inplace=True)
df_mkt_player_team_matches_goals.drop('team_api_id_y', axis=1, inplace=True)
df_mkt_player_team_matches_goals.rename(columns={'player_name_x':'player_name'}, inplace=True)
df_mkt_player_team_matches_goals.rename(columns={'team_api_id_x':'team_api_id'}, inplace=True)

In [100]:
columns_type_object = list(df_mkt_player_team_matches_goals.select_dtypes(include='object').columns)
for col in columns_type_object:
    df_mkt_player_team_matches_goals[col] = df_mkt_player_team_matches_goals[col].astype('string')

In [101]:
df_mkt_player_team_matches_goals.sort_values(by=['season','player_api_id'], inplace=True)

In [102]:
df_mkt_player_team_matches_goals.groupby(['player_api_id','season'])['id'].count()

player_api_id  season
10967          2011       2
               2012       1
11685          2009       2
               2010       3
               2011      10
                         ..
683450         2015       1
688295         2015       2
690188         2015       3
696365         2015       5
722766         2015       1
Name: id, Length: 1428, dtype: int64

Y ahora <b>vamos a hacer el merge equivalente al anterior, pero para las tarjetas</b> entre 'df_mkt_player_team_matches' con 'df_cards_players_teams' en base al player_api_id y la  temporada de referencia.

In [103]:
df_mkt_player_team_matches_cards = pd.merge(df_mkt_player_team_matches, df_cards_players_teams, \
        on=['player_api_id','season'], how='inner')

In [104]:
df_mkt_player_team_matches_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7555 entries, 0 to 7554
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   player_name_x      7555 non-null   object        
 1   player_api_id      7555 non-null   int64         
 2   player_nation      7555 non-null   object        
 3   season             7555 non-null   int64         
 4   team_long_name     7555 non-null   object        
 5   team_api_id_x      7555 non-null   int64         
 6   season_player_age  7555 non-null   int64         
 7   market_val_amnt    7555 non-null   int64         
 8   player_pos         7555 non-null   object        
 9   id                 7555 non-null   int64         
 10  match_api_id       7555 non-null   int64         
 11  country            7555 non-null   string        
 12  elapsed            7555 non-null   int64         
 13  card_type          7555 non-null   string        
 14  elapsed_

In [105]:
# Eliminamos algunas columnas redundantes y renombramos otras (lo mismo que con el otro DF)
df_mkt_player_team_matches_cards.drop('player_name_y', axis=1, inplace=True)
df_mkt_player_team_matches_cards.drop('team_api_id_y', axis=1, inplace=True)
df_mkt_player_team_matches_cards.rename(columns={'player_name_x':'player_name'}, inplace=True)
df_mkt_player_team_matches_cards.rename(columns={'team_api_id_x':'team_api_id'}, inplace=True)

In [106]:
columns_type_object = list(df_mkt_player_team_matches_cards.select_dtypes(include='object').columns)
for col in columns_type_object:
    df_mkt_player_team_matches_cards[col] = df_mkt_player_team_matches_cards[col].astype('string')

In [107]:
df_mkt_player_team_matches_cards.sort_values(by=['season','player_api_id'], inplace=True)

In [108]:
df_mkt_player_team_matches_cards.groupby(['player_api_id','season'])['id'].count()

player_api_id  season
10967          2011      2
               2012      1
11027          2009      1
11685          2009      2
               2010      4
                        ..
671644         2015      3
683450         2015      1
688295         2015      1
690188         2015      2
722766         2015      2
Name: id, Length: 1958, dtype: int64

## ESTUDIO EXPLORATORIO

Vemos la información de las temporadas

In [109]:
print("Número de temporadas:" ,len(df_mkt_player_team_matches['season'].unique()))
print("Más antigua con datos: ", min(df_mkt_player_team_matches['season'].unique()))
print("Última con datos: ", max(df_mkt_player_team_matches['season'].unique()))

Número de temporadas: 13
Más antigua con datos:  2009
Última con datos:  2021


Preparamos un Dataframe con el número de jugadores por temporada

In [110]:
players_season_list = [] 
df_for_altair = pd.DataFrame(index=['numero_jugadores'])    
for i in np.arange(2009,2022):
    count = len(df_mkt_player_team_matches.loc[df_mkt_player_team_matches['season']==i]\
        .groupby(['player_api_id']))
    df_for_altair[str(i)] = count    

Mostramos con Altair nuestro primer gráfico: el número de jugadores distintos de que disponemos
por temporada, otra cosa es que en cada temporada, un jugador cambie de equipo alguna vez, o se 
realiza una actualización de los atributos futbolísticos. 

Vemos como en las 5 últimas temporadas(2017-2021) no hay una gran cantidad de datos significativos, al principio ya se comentó que los datos recogidos son basicamente de entre 2008/2009 y 2015/2016

In [111]:
import altair as alt

alt.Chart(df_for_altair.T.reset_index()).mark_line().encode(
      alt.X('index:N',title="Año"),
      alt.Y('numero_jugadores:Q',title="Número de Jugadores" )
).properties(
    width=500)

alt.Chart(...)

Preparamos un segundo gráfico exploratorio para mostrar el valor acumulado del mercado por año y
verlo en cada año repartido por la nacionalidad de los futbolistas. 
Tenemos que partir el Dataframe para que Altair nos permita mostrar el gráfico, ya que hay un límite de filas.

In [112]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [113]:
df_altair2 = df_mkt_player_team_matches.loc[:,['player_name','player_nation', 'season', 'market_val_amnt']]
df_altair2.drop_duplicates(inplace=True)
df_altair2.dropna(axis=0, how='any', subset=['market_val_amnt'], inplace=True)
df_altair2.sort_values(by=['season'])
df_altair2['market_val_amnt'] = df_altair2['market_val_amnt'].astype('string')
df_altair2['tooltip'] = df_altair2['player_name'] + ' from '+ df_altair2['player_nation'] \
    + ', Valor de mercado: ' + df_altair2['market_val_amnt']
df_altair2['market_val_amnt'] = df_altair2['market_val_amnt'].astype('int64')

Vemos en el gráfico como el precio de mercado es muy alto por fultbolistas franceses, brasileños o croatas por ejemplo.

In [114]:
import altair as alt

alt.Chart(df_altair2).mark_bar().encode(
      alt.X('season:N', title='Año'),
      alt.Y('market_val_amnt:Q', title='Valor de mercado acumulado desglosado por pais', scale=alt.Scale(domain = [0, 4e9])),
      color='player_nation',
      tooltip='tooltip',
).properties(
    width=800, height=650).interactive()

alt.Chart(...)

Este gráfico tiene algunos problemas, uno es la idea equivocada de hacer parecer que el valor de mercado global va aumentando entre el 2009 y el 2015 y después practicamente desaparece; como ya hemos comentado , todo está en función de los datos que disponemos, solo tenemos información de jugadores entre los años 2009 y 2016, sólo hay que fijarse en el gráfico anterior para ver que éste está en función del otro.

<h1>GRÁFICOS DE OBJETIVOS </h1>

## 1. RENDIMIENTO MEDIO GENERAL DE LOS FUTBOLISTAS

Ahora vamos a mostrar un gráfico de dispersión por cada año de entre los que tengamos suficientes datos: es decir, del 2009 al 2016.
En el Eje X se va a mostrar el rendimiento medio de un futbolista en un año, mientras que en el eje Y se mostrará el valor de mercado medio de ese futbolista en ese año. 
El color indica la edad del futbolista, cuanto más rojizo, mayor edad. El tooltip muestra el nombre del futbolista y el club en el que se ha incorporado en ese año.
El indicador de rendimiento cumple la función de ser una medida totalmente general en cuanto al puesto en el que se desempeña el jugador.

Así pues, preparamos un nuevo gráfico para empezar a relacionar el valor de mercado con el rendimiento que ofrecen esos futbolistas en un momento dado, son los campos <b>'market_val_amnt'</b> y <b>'overall_rate'</b> los que comparamos en cada jugador por temporada.

In [ ]:
df_altair3 = df_mkt_player_team_matches_attributes.loc[:,['player_name','season','season_player_age','team_long_name', \
                                                              'market_val_amnt', 'overall_rating']]
df_altair3.drop_duplicates(inplace=True)
df_altair3.dropna(axis=0, how='any', inplace=True)

df_altair3['tooltip'] = df_altair3['player_name'] + ' playing in '+ df_altair3['team_long_name']
df_altair3.info()

print('\nMínimo overall rating: ',df_altair3['overall_rating'].min())
print('Máximo overall rating: ',df_altair3['overall_rating'].max())
print('\nMínimo market val.: ',df_altair3['market_val_amnt'].min())
print('Máximo market val.: ',df_altair3['market_val_amnt'].max())

In [ ]:
import altair as alt

In [ ]:
sca_list = None
for i in range(2009,2017):
    sca=alt.Chart(df_altair3).mark_point().encode(
          alt.X('mean(overall_rating):Q', title="Rendimiento medio jugador en ese año", scale=alt.Scale(domain=[45,100])),
          alt.Y('mean(market_val_amnt):Q', title="Valor de mercado", axis=alt.Axis(titleAngle=-65, titlePadding=35)),
          tooltip='tooltip',
          color=alt.Color('season_player_age:Q', scale=alt.Scale(scheme='goldred'), 
                          legend=alt.Legend(orient="top", direction='horizontal', 
                                            titleAnchor='middle',  title="Edad del jugador"))
    ).transform_filter(
        'datum.season=='+str(i)
    ).properties( title = str(i) ).interactive()
    
    if sca_list is None:
        sca_list = sca
    else:    
        sca_list |= sca
sca_list

## 2. DELANTEROS Y GOLES

En el siguiente gráfico vamos a medir <b>la capacidad goleadora de los delanteros</b>, así que primero vamos a filtrar los jugadores cuya demarcación hace que se espere de ellos una cierta cantidad de goles.

In [ ]:
df_altair4_only_attackers = df_mkt_player_team_matches_goals.loc[df_mkt_player_team_matches_goals['player_pos'] \
                                            .isin(['AM', 'LW', 'RW', 'CF', 'SS', 'attack'])]

In [ ]:
df_altair4_only_attackers.info()

In [ ]:
df_altair4 = df_altair4_only_attackers.loc[:,['player_api_id','player_name','season','season_player_age' \
                                                     ,'market_val_amnt']].sort_values(by=['season','player_api_id'])
df_altair4.drop_duplicates(inplace=True)
df_altair4.dropna(axis=0, how='any', inplace=True)

season_goals = df_altair4_only_attackers.groupby(['season','player_api_id'])['id'].count()
season_goals = season_goals.reset_index(level=[0,1])

In [ ]:
df_altair4.info()

In [ ]:
season_goals.info()

In [ ]:
df_altair4 = pd.merge(df_altair4, season_goals, on=['player_api_id', 'season'])

In [ ]:
df_altair4.head(10)

In [ ]:
df_altair4.rename(columns={'id':'goals'}, inplace=True)
df_altair4['goals'] = df_altair4['goals'].astype('string')
df_altair4['market_val_amnt'] = df_altair4['market_val_amnt'].astype('string')
df_altair4['tooltip'] = df_altair4['player_name'] +', '+ df_altair4['goals'] +' goles\n'+'Valor de mercado: '\
                        + df_altair4['market_val_amnt']
df_altair4['goals'] = df_altair4['goals'].astype('uint8')
df_altair4['market_val_amnt'] = df_altair4['market_val_amnt'].astype('int64')

In [ ]:
df_altair4.head(10)

In [ ]:
scatter_all2 = None

for i in range(2009,2016):
    scatter2=alt.Chart(df_altair4).mark_point().encode(
          #alt.Y('player_name:N'),
          alt.X('mean(market_val_amnt):Q', title="Valor de mercado", axis=alt.Axis(titlePadding=5)),
          alt.Y('goals:Q', title="Goles jugador en ese año", sort='ascending'),          
          tooltip='tooltip',
          color=alt.Color('season_player_age:Q', scale=alt.Scale(scheme='goldred'), 
                          legend=alt.Legend(orient="top", direction='horizontal', 
                                            titleAnchor='middle',  title="Edad del Jugador"))
    ).transform_filter(
        'datum.season=='+str(i)
    ).properties(title = str(i) ).interactive()
    
    
    if scatter_all2 is None:
        scatter_all2 = scatter2
    else:    
        scatter_all2 |= scatter2
        
scatter_all2

## 3. DEFENSAS Y TARJETAS

En el siguiente gráfico, por el contrario vamos a medir la agresividad de los defensas, cuantas tarjetas amarillas reciben y cual es el riesgo de que les expulsen por tarjeta roja .

In [ ]:
df_altair5_only_defences = df_mkt_player_team_matches_cards.loc[df_mkt_player_team_matches_cards['player_pos'] \
                                            .isin(['RB', 'CB', 'LB', 'DM', 'defence'])]

In [ ]:
df_altair5_only_defences.info()

In [ ]:
df_altair5 = df_altair5_only_defences.loc[:,['player_api_id','player_name','season','season_player_age' \
                                                     ,'market_val_amnt']].sort_values(by=['season','player_api_id'])
df_altair5.drop_duplicates(inplace=True)
df_altair5.dropna(axis=0, how='any', inplace=True)

season_cards = df_altair5_only_defences.groupby(['season','player_api_id', 'card_type'])[['id']].count()

season_cards = season_cards.reset_index(level=[0,1,2])

In [ ]:
df_altair5.info()

In [ ]:
season_cards.info()

In [ ]:
df_altair5 = pd.merge(df_altair5, season_cards, on=['player_api_id', 'season'])

In [ ]:
df_altair5.info()

In [ ]:
df_altair5['card_type'].replace('y','Amarilla', inplace=True)
df_altair5['card_type'].replace('y2','Roja', inplace=True)
df_altair5['card_type'].replace('r','Roja', inplace=True)

In [ ]:
df_altair5.rename(columns={'id':'cards'}, inplace=True)
df_altair5['cards'] = df_altair5['cards'].astype('string')
df_altair5['market_val_amnt'] = df_altair5['market_val_amnt'].astype('string')
df_altair5['tooltip'] = df_altair5['player_name'] +', '+ df_altair5['cards'] +' tarjeta(s) ' +df_altair5['card_type']
df_altair5['cards'] = df_altair5['cards'].astype('uint8')
df_altair5['market_val_amnt'] = df_altair5['market_val_amnt'].astype('int64')

In [ ]:
df_altair5.head(10)

In [ ]:
scatter_all3 = None

for i in range(2009,2016):
    scatter3=alt.Chart(df_altair5).mark_square().encode(
          #alt.Y('player_name:N'),
          alt.X('mean(market_val_amnt):Q', title="Valor de mercado", axis=alt.Axis(titlePadding=5)),
          alt.Y('cards:Q', title="Tarjetas jugador en ese año", sort='ascending', axis=alt.Axis(tickCount=20)),          
          tooltip='tooltip',
          color=alt.Color('card_type:O', scale=alt.Scale(scheme='yelloworangered'), 
                          legend=alt.Legend(orient="top", direction='horizontal', 
                                            titleAnchor='middle',  title="Color Tarjeta"))
    ).transform_filter(
        'datum.season=='+str(i)
    ).properties(title = str(i) , width=600).interactive()
    
    
    if scatter_all3 is None:
        scatter_all3 = scatter3
    else:    
        scatter_all3 |= scatter3
        
scatter_all3.configure_point(
        size=300,
        color='lightyellow',
        filled=False
    )